# Making a goto.ma from an OpenCPN waypoint list
I recommend using the .py instead of this file, but in case anyone wants a .ipynb instead of a .py I left it here

In [1]:
import numpy as np                  # number processing
from datetime import datetime       # for goto file timestamp 
import pandas as pd                 # data processing
import folium                       # easy maps, also interactive
from folium.features import DivIcon # needed for map labels
from tkinter.filedialog import askopenfilename

# filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file

In [3]:
# import, remove header, and pick columns I want
# filename = 'endurance.csv'
wantcol = ['Leg','To waypoint','Distance','Bearing','Latitude','Longitude']

filename = 'endurance_nonan.csv'

raw = pd.read_csv('../routes/data/{}'.format(filename),
                  header=9,
                  usecols=wantcol,
                  index_col=0,
                  sep=',',
                  )

raw['To waypoint'].fillna('unnamed',inplace=True) # replace nan values with a string so waypoint name looping doesn't break

# make loop variable
howmanytoloop = raw.shape[0]
loopnum = range(0,howmanytoloop)

# clean data to proper format
lat = []
lon = []
bear = []
dist = []
name = []
latmap = []
lonmap = []

for i in loopnum : 
    
    # latitude strings
    latdegstr = raw['Latitude'][i][0:2]
    latminstr = raw['Latitude'][i][4:8]
    latns     = raw['Latitude'][i][10]

    # latitude strings to integers for mapping later
    latmaploop = int(float(latdegstr)) + ((int(float(latminstr)))/60)
    if latns == 'S':
        latmaploop = latmaploop * -1
    latmap.append(latmaploop)

    # put strings together into goto10 format
    latloop = latdegstr + latminstr            # degrees + minutes.decimalminutes
    if latns == 'S':                           # check if in southern hemisphere
        latloop = '-' + latloop                # if so make number negative
    lat.append(latloop)                        # append result to lat

   # longitude strings
    londegstr = raw['Longitude'][i][0:3]
    lonminstr = raw['Longitude'][i][5:9]
    lonew     = raw['Longitude'][i][11]

    # longitude strings to integers for mapping later
    lonmaploop = int(float(londegstr)) + ((int(float(lonminstr)))/60)
    if lonew == 'W':
        lonmaploop = lonmaploop * -1
    lonmap.append(lonmaploop)

    lonloop = londegstr + lonminstr        # degrees + minutes.decimalminutes
    if lonew == 'W':                       # check if western
        lonloop = '-' + lonloop            # if so make number negative
    lon.append(lonloop)                    # append result to lon

    bearloop = raw['Bearing'][i][0:3]      # bearing between points 
    bear.append(bearloop)                  # append result

    nameloop = raw['To waypoint'][i][0:]   # name of waypoint
    name.append(nameloop)                  # append

    distloop = raw['Distance'][i][0:3]     # distance between points 
    distloop = int(float(distloop))        # convert string to float and make distloop = 
    distloop = (distloop * (1.852))        # convert nautical miles to kilometer 
    dist.append(distloop)                  # append

# put the above into final dataframe
coord_list = pd.DataFrame()
coord_list['lat'] = lat
coord_list['lon'] = lon
coord_list['name'] = name
coord_list['bearing (T)'] = bear
coord_list['dist (km)'] = dist


In [4]:
# save waypoint list as interactive HTML via Folium

mapcenter = [(np.max(latmap)+np.min(latmap))/2, (np.max(lonmap)+np.min(lonmap))/2] 

f = folium.Figure(width=200, height=200)

map = folium.Map( 
    location=mapcenter,
    zoom_start=8,
    control_scale=True,
    zoom_control=True,
    scrollWheelZoom=True,
    dragging=True)

# add markers to map
for i in loopnum : 
    folium.Marker([latmap[i],lonmap[i]], popup='{}').add_to(map) # add buoy marker to map
    folium.map.Marker(                                    # add text
        [latmap[i],lonmap[i]],
        icon=DivIcon(
            icon_size=(30,30),
            icon_anchor=(0,0),
            html='<div style="color:#ffffff;font-size: 12pt">{}</div>'.format(coord_list['name'][i]),
            )
        ).add_to(map)

line_between =[]
for i in loopnum :
    oneline = [latmap[i],lonmap[i]]
    line_between.append(oneline)

line=folium.PolyLine(
    locations=line_between,
    weight=3,
    color='#e02525'
    ).add_to(map)

map.save('../routes/{}.html'.format(filename[0:-4]))
map

In [5]:
# Export a properly formatted goto10.ma file

savepath='../routes/'
radius = '100'
glider_name = 'namegoeshere'

# for use if asking user for input
   # glider_name = input('Glider Name [000] : ')
   # glider_name = glider_name or '000'

   # radius = input('satisfying radius(m)  [100] : ')
   # radius = radius or '100'

# From Joe's KML2Goto.py
GotoFile = open(savepath+'goto_l10.ma', 'w')

GotoFile.write('behavior_name=goto_list \n')
GotoFile.write('#============================================================ \n')
GotoFile.write('# --- goto_l10.ma\n')
GotoFile.write('# Generated for {} by OpenCPN2goto.py from {} at {} \n'.format(glider_name,filename,datetime.now().strftime('%b %d %Y %H:%M:%S')))   
GotoFile.write('# {}m Satisfying Radius\n'.format(radius))
GotoFile.write('#============================================================\n')
GotoFile.write('\n')
GotoFile.write('<start:b_arg>\n')
GotoFile.write('    b_arg: num_legs_to_run(nodim)   -1    # loop through waypoints\n')
GotoFile.write('    b_arg: start_when(enum)          0    # BAW_IMMEDIATELY\n')
GotoFile.write('    b_arg: list_stop_when(enum)      7    # BAW_WHEN_WPT_DIST\n')
GotoFile.write('\n')
GotoFile.write('    # SATISFYING RADIUS\n')
GotoFile.write('      b_arg:  list_when_wpt_dist(m)  {}\n'.format(radius))
GotoFile.write('\n')
GotoFile.write('    # LIST PARAMETERS                  LON     LAT\n')
GotoFile.write('      b_arg: initial_wpt(enum)       {}    # First wpt in list\n'.format(lon[0] + '    ' + lat[0]))
GotoFile.write('      b_arg: num_waypoints(nodim)    {}                # Number of waypoints in list\n'.format(len(coord_list)))
GotoFile.write('<end:b_arg>\n')
GotoFile.write('\n')
GotoFile.write('<start:waypoints>\n')
GotoFile.write('#     LON     LAT      \u00b0T         name\n')
for i in loopnum:
   GotoFile.write('    {}   # {}  |  {}\n'.format((lon[i] + '    ' + lat[i]),(coord_list['bearing (T)'][i]),coord_list['name'][i]))

GotoFile.write('<end:waypoints>')
GotoFile.close()